There i parse telegram chat history 

In [16]:
!pip install lxml

     ---------------------------------------- 3.6/3.6 MB 1.7 MB/s eta 0:00:00


You should consider upgrading via the 'C:\Users\royta\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [101]:
import os
import shutil
import pickle
from lxml import html
from contextlib import suppress
from dataclasses import dataclass

import torch
import numpy as np
from PIL import Image
from facenet_pytorch import MTCNN, InceptionResnetV1

In [16]:
def get_name(body: html.HtmlElement) -> str|None:
    with suppress(Exception):
        name = body.find_class('from_name')[0].text
        name = name.replace('\n', '').strip()
        return name

In [17]:
def get_photo(body: html.HtmlElement) -> str|None:
    with suppress(Exception):
        div = body.find_class('media_wrap')[0]
        a = div.getchildren()[0]
        photo = a.attrib['href']
        return photo

In [18]:
def get_text(body: html.HtmlElement, text="") -> str|None:
    with suppress(Exception):
        div = body.find_class('text')[0]
        for t in div.itertext():
            text+= t
            
        text = text.replace('\n', '', 2).strip()
        return text

In [19]:
messages = []

for catalog in os.listdir('tg_data'):
    
    for file in os.listdir('tg_data/' + catalog):
        if '.html' in file:

            with open(f"tg_data/{catalog}/{file}", "r", encoding='utf-8') as f:
                _html = f.read()

            tree = html.fromstring(_html)
            history = tree.xpath("/html/body/div/div[2]/div")[0]
            history = history.find_class('message default clearfix')

            for message in history:
                body = message.find_class('body')[0]
                body = message.find_class('body')[0]
                    
                photo = get_photo(body) 
                photo = f"{catalog}/{photo}" if photo else None
                messages+= [{
                    'name': get_name(body),
                    'text': get_text(body),
                    'photo': photo,
                }]

In [22]:
mtcnn = MTCNN()
model1 = InceptionResnetV1(pretrained='vggface2').eval()
model2 = InceptionResnetV1(pretrained='casia-webface').eval()

100%|████████████████████████████████████████████████████████████████████████████████| 111M/111M [02:53<00:00, 667kB/s]


In [102]:
embeds_models = [model1, model2]

In [124]:
class PhotoEmbedingStorage:
    """This storage class contains 
    primary photo path+name: generated unique id 
    generated unique id: [model1 embeddings, ..., modelN embeddings]
    """
    def __init__(self, path=None):
        self.path = path
        self.path_id: dict[str, int] = {}
        self.id_path: dict[int, str] = {}
        self.id_embs: dict[int, np.ndarray] = {}
        
        if path:
            self.load()
    
    def save(self, path=None):
        "Save storage to path"
        if path is None:
            if self.path:
                path = self.path
            else:
                raise Exception('Provide path to save')

        with open(path, 'wb') as f:
            pickle.dump({
                'path_id': self.path_id,
                'id_path': self.id_path,
                'id_embs': self.id_embs,
            }, f)
            
                
    def load(self, path=None):
        "Load storage to path"
        if path is None:
            if self.path:
                path = self.path
            else:
                raise Exception('Provide path to save')
    
        with open(path, 'rb') as f:
            loaded_dict = pickle.load(f)
            self.path_id = loaded_dict['path_id']
            self.id_path = loaded_dict['id_path']
            self.id_embs = loaded_dict['id_embs']
            
            
    def __generate_id(self) -> int:
        return len(self.path_id)
    
    
    def __setitem__(self, key: str, value: list[np.ndarray|torch.Tensor]):
        """Put embeddings in storage
        :param key: primary photo path+name
        :param value: list with embeddings from models
        """
        assert isinstance(key, str)
        assert isinstance(value, list)
        
        for emb in value:
            assert isinstance(emb, (np.ndarray, torch.Tensor))
        
        uid = self.__generate_id()
        self.path_id[key] = uid
        self.id_path[uid] = key
        self.id_embs[uid] = embs
        
    
    def __getitem__(self, item: tuple[int|str, int]) -> np.ndarray:
        """Get embeddings from storage
        if item is tuple, there are key and n
        :param key: id or primary path
        :param n: since we have different embeddings
            from different models, this argument responsible 
            for number of returned embedding, N - embedding from N+1 model.
            default: 0
        """
        if isinstance(item, tuple):
            key, n = item
        else:
            key, n = item, 0
        
        if isinstance(key, int):
            return self.id_embs[key][n]
        
        elif isinstance(key, str):
            uid = self.path_id[key]
            return self.id_embs[uid][n]
        
        else:
            raise TypeError(f'Dont understand key type: {type(key)}')
            

    def __len__(self) -> int:
        return len(self.path_id)

In [125]:
embeder = PhotoEmbedingStorage()

In [126]:
def process_photo(photo_path:str, saved_path:str):
    img = Image.open(photo_path)
    img_cropped = mtcnn(img, save_path='tmp.png').unsqueeze(0)

    embs = [model(img_cropped) for model in embeds_models]
    embeder[photo_path] = embs

    uid = embeder.path_id[photo_path]
    shutil.move('tmp.png', f'{saved_path}/{uid}.png')

In [60]:
with torch.no_grad():
    for current, previous in zip(messages[1:], messages[:-1]):
        if previous['photo']:
            photo_path = 'tg_data/' + previous['photo']

            if current['text'] in ['💌', '❤️', '👍']:
                saved_path = 'data/target'

            elif current['text'] in ['👎']:
                saved_path = 'data/opposite'

            else:
                continue
        
            
            process_photo(photo_path, saved_path)

In [ ]:
embeder.save('emb storage.pkl')

In [9]:
DATASET_PATH = "data"

In [ ]:
def procces_markup(src, dst)

In [8]:
for file in os.listdir('markup/test_target'):
    photo_path = 'markup/test_target/' + file
    photo_name = file
    saved_path = DATASET_PATH + "/test_target/" + file
    
    img = Image.open(photo_path)
    _, e = file.split('.')
    if e.lower() == 'png':
        img = img.convert('RGB')
        
    img_cropped = mtcnn(img, save_path=saved_path)
    
    
for file in os.listdir('markup/test_opposite'):
    photo_path = 'markup/test_opposite/' + file
    photo_name = file
    saved_path = DATASET_PATH + "/test_opposite/" + file

    img = Image.open(photo_path)
    _, e = file.split('.')
    if e.lower() == 'png':
        img = img.convert('RGB')
        
    img_cropped = mtcnn(img, save_path=saved_path)
    
    
for file in os.listdir('markup/target'):
    photo_path = 'markup/target/' + file
    photo_name = file
    saved_path = DATASET_PATH + "/markup_target/" + file
    
    img = Image.open(photo_path)
    _, e = file.split('.')
    if e.lower() == 'png':
        img = img.convert('RGB')
        
    img_cropped = mtcnn(img, save_path=saved_path)
    
    
for file in os.listdir('markup/opposite'):
    photo_path = 'markup/opposite/' + file
    photo_name = file
    saved_path = DATASET_PATH + "/markup_opposite/" + file
    
    img = Image.open(photo_path)
    _, e = file.split('.')
    if e.lower() == 'png':
        img = img.convert('RGB')
        
    img_cropped = mtcnn(img, save_path=saved_path)

C:\Users\royta\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\nn\functional.py:780: UserWarning: Note that order of the arguments: ceil_mode and return_indices will changeto match the args list in nn.MaxPool2d in a future release.
  warnings.warn("Note that order of the arguments: ceil_mode and return_indices will change"


Manually clear the dataset